# Data Preparation
In this notebook we will prepare our data for our search function to use.

Currently we have data stored in four different csv files.

 1.dirty_data.csv
 
 2.missing_data.csv
 ### It can be computationally expensive to produce analysis results from multiple data-sources for incomming stream of requests.
So we will prepare our data and save it in an easily searchable structure.

In [59]:
# Import the needed modules...
import pandas as pd
from collections import defaultdict
from os import getcwd

# Define Paths to data files.

In [60]:
PATH_dirty_data   = f"{getcwd()}/dataStore/dirty_data.csv"
PATH_missing_data  = f"{getcwd()}/dataStore/missing_data.csv"

# Data Engineering
Get data in dataframes.

Convert data to a single dictionary.

In [61]:
"""
    Read data from movies.csv
"""
df_dirty_data            = pd.read_csv(PATH_dirty_data)
dirty_data_table_columns = df_dirty_data.columns.tolist()
print(f"COLUMNS : {dirty_data_table_columns}")

COLUMNS : ['order_id', 'customer_id', 'date', 'nearest_warehouse', 'shopping_cart', 'order_price', 'delivery_charges', 'customer_lat', 'customer_long', 'coupon_discount', 'order_total', 'season', 'is_expedited_delivery', 'distance_to_nearest_warehouse', 'latest_customer_review', 'is_happy_customer']


In [62]:
"""
    Read data from missing_data.csv
"""
df_missing_data            = pd.read_csv(PATH_missing_data)
missing_data_table_columns = df_missing_data.columns.tolist()
print(f"COLUMNS : {missing_data_table_columns}")

COLUMNS : ['order_id', 'customer_id', 'date', 'nearest_warehouse', 'shopping_cart', 'order_price', 'delivery_charges', 'customer_lat', 'customer_long', 'coupon_discount', 'order_total', 'season', 'is_expedited_delivery', 'distance_to_nearest_warehouse', 'latest_customer_review', 'is_happy_customer']


### order_Id is a common column in all two tables so we will use it as a primary search-key
### A user will always search a order by its customer_id so we will create a Global secondary index to be able to perform search our datastore.
it will obviously take some extra space but almost negligible as compared to the size of the original data.
In addition, It will make our searching faster and efficient so it's a good deal.

In [63]:
print(f"It is {pd.Series(df_dirty_data['order_id']).is_unique}  that the column 'order_id' has unique values for all entries in dirty_data dataframe.")
print(f"It is {pd.Series(df_missing_data['order_id']).is_unique} that the column 'order_id' has unique values for all entries in missing_data dataframe.")

# Sort dirty_data dataframe on the basis of order_Id as order_Id is unique for all entries...
df_dirty_data_sorted = df_dirty_data.sort_values(by=['order_id'])

# Sort missing_data dataframe on the basis of order_Id as order_Id is unique for all entries...
df_missing_data_sorted  = df_missing_data.sort_values(by=['order_id'])

It is True  that the column 'order_id' has unique values for all entries in dirty_data dataframe.
It is True that the column 'order_id' has unique values for all entries in missing_data dataframe.


In [64]:
# from dirty_data dataframe...
order_Id                         = df_dirty_data_sorted["order_id"].tolist()
customer_Id                      = df_dirty_data_sorted["customer_id"].tolist()
shoppingCart = [shoppingCart.split("|") for shoppingCart in df_dirty_data["shopping_cart"].tolist()]

# from missing_data dataframe...
latest_customer_review           = df_missing_data_sorted["latest_customer_review"].tolist()
is_happy_customer                = df_missing_data_sorted["is_happy_customer"].tolist()



In [65]:
dirty_dataDict             = {}
global_secondaryIndex      = {}
for idx, order_id in enumerate(order_Id):
    dirty_dataDict[order_id] = {
        "Cart" :shoppingCart[idx],
        "missing_data" : {
                "latest_customer_review"     : latest_customer_review[idx],
                "is_happy_customer"          : is_happy_customer[idx]

        }
    }
    
    global_secondaryIndex[customer_id[idx]] = customer_id


In [66]:
# delete veriables which are no longer in use while holding large amount of data.
#del order_Id
#del customer_Id
#del shoppingCart
#del latest_customer_review
#del is_happy_customer


In [68]:
import json
print("[INFO] Writing dirty_data Data into the disk...")
with open('dataStore/dataFinal.json', 'w') as fp:
    json.dump(dirty_dataDict, fp, sort_keys=True, indent=4)
print("[INFO] Writing Global Secondary Index Data into the disk...")
with open('dataStore/dataFinal_GIS.json', 'w') as fp:
    json.dump(global_secondaryIndex, fp, sort_keys=True, indent=4)

[INFO] Writing dirty_data Data into the disk...
[INFO] Writing Global Secondary Index Data into the disk...
